In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset
import openpyxl as op
from tqdm.auto import tqdm
import plotly.graph_objs as go
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", message="No frequency information was")
warnings.filterwarnings("ignore", message="Maximum Likelihood optimization failed to ")

instances = pd.read_csv("../instances/instances.csv", index_col=0).to_numpy().reshape(14)

In [2]:
INSTANCE = 0

In [3]:
data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[INSTANCE]))
data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
data_series.index = data_series['local_15min']
data_series.drop(columns=['local_15min'], inplace=True)
data_demand = data_series[['nonshiftable']]

In [4]:
data_demand_hourly = data_demand.resample("H").mean()
data_demand_hourly.head()

,nonshiftable
local_15min,
2019-05-01 00:00:00,0.065125
2019-05-01 01:00:00,0.043813
2019-05-01 02:00:00,0.067750
2019-05-01 03:00:00,0.099062
2019-05-01 04:00:00,0.177687


In [19]:
start_date = "2019-05-01"
end_date = "2019-07-01"
training_data = data_demand_hourly[(data_demand_hourly.index >= start_date) & (data_demand_hourly.index<end_date)]

In [8]:
def sarima_model(training_data, order, seasonal_order, timesteps, days):
    
    k = timesteps*days

    # Inicializar df con la información
    df = pd.DataFrame()

    # Calcular cantidad de días a pronósticar
    days_forecast = training_data.index[-1]-training_data.index[k-1]
    days_forecast = days_forecast.days

    for i in range(days_forecast):
        
        # Training set
        train = training_data[timesteps*i:k+timesteps*i]
        model = sm.tsa.SARIMAX(train['nonshiftable'].asfreq("60min"), order = order,seasonal_order=seasonal_order, freq=train.index.inferred_freq).fit(disp=False)

        for j in range(timesteps):

            # Prediction
            forecast_dates=[train.index[-1]+ DateOffset(minutes=60*j) + DateOffset(minutes=60*x)for x in range(1,timesteps+1-j)]
            df_forecast = training_data.loc[forecast_dates].copy()
            forecast = model.get_prediction(start = forecast_dates[0], end = forecast_dates[-1])
            forecast_mean = forecast.predicted_mean
            forecast_mean.index = forecast_dates
            forecast_U = forecast.predicted_mean + forecast.se_mean
            forecast_L = forecast.predicted_mean - forecast.se_mean
            forecast_U.index = forecast_dates
            forecast_L.index = forecast_dates

            # MSE
            df_forecast['forecast'] = forecast_mean
            #df_forecast['MSE'] = (df_forecast['forecast']-df_forecast['p'])**2
            df_forecast['Upper bound'] = forecast_U
            df_forecast['Lower bound'] = forecast_L

            # Update model with new data
            new_data = train.index[-1]+DateOffset(minutes=60*(j+1))
            model = model.extend(training_data['nonshiftable'][new_data:new_data].asfreq("60min"))

            # Update dataframe
            df_forecast['From'] = train.index[-1] + DateOffset(minutes=60*j)
            df_forecast['Day'] = train.index[-1] + DateOffset(minutes=60)
            df = pd.concat([df, df_forecast.reset_index(names='local_15min')])


        #print(str(i)+"|"+str(days_forecast-1),'train:', train.index[0], '->', train.index[-1], '|forecast:', forecast_dates[0], '->', forecast_dates[-1], '|mse:', df_forecast['MSE'].mean())

    return df

In [7]:
def auto_sarima(training_data, order_limits, timesteps, days, ruta):

    # Crear libro de excel
    book = op.Workbook()
    sheet = book.active
    sheet.title = 'Resultados'
    sheet.cell(3,1).value = 'p'
    sheet.cell(3,2).value = 'd'
    sheet.cell(3,3).value = 'q'
    sheet.cell(3,4).value = 'sp'
    sheet.cell(3,5).value = 'sd'
    sheet.cell(3,6).value = 'sq'
    sheet.cell(3,7).value = 's'
    sheet.cell(3,8).value = 'mse_mean'
    sheet.cell(3,9).value = 'mse_sd'

    # Inicializar lista con el mse de cada modelo y el contador
    mse_models = {}
    c = 3
    for p in order_limits['p']:
        for d in order_limits['d']:
            for q in order_limits['q']:
                for s in order_limits['s']:
                    for sp in order_limits['sp']:
                        for sd in order_limits['sd']:
                            for sq in order_limits['sq']:

                                if(s == 0 and sp+sd+sq!=0):
                                    continue
                          
                            
                                #print(" Model:", "({}, {}, {}), s({}, {}, {}, {})".format(p,d,q,sp,sd,sq,s))
                                time.sleep(0.1)
                                df = sarima_model(training_data=training_data, order=(p,d,q), seasonal_order=(sp,sd,sq,s), timesteps=timesteps, days=days)
                                df['mse'] = (df['nonshiftable']-df['forecast'])**2
                                mse_mean = df['mse'].mean()
                                #print(mse_mean)
                                mse_sd = df['mse'].var()**(1/2)
                                mse_models["({}, {}, {}), s({}, {}, {}, {})".format(p,d,q,sp,sd,sq,s)] = mse_mean
                                c = c+1
                                sheet.cell(c,1).value = p
                                sheet.cell(c,2).value = d
                                sheet.cell(c,3).value = q
                                sheet.cell(c,4).value = sp
                                sheet.cell(c,5).value = sd
                                sheet.cell(c,6).value = sq
                                sheet.cell(c,7).value = s
                                sheet.cell(c,8).value = mse_mean
                                sheet.cell(c,9).value = mse_sd

                                book.save(ruta)

    # best model
    sheet.cell(1,1).value = 'Mejor modelo:'
    sheet.cell(1,2).value = min(mse_models, key=mse_models.get)
    sheet.cell(1,3).value = min(mse_models.values())

    book.save(ruta)

In [9]:
# Parámetros
timesteps = 24
days = 31 # 5 dias de observación
ruta = 'resultados/60min/sarima/sarima2_i{}.xlsx'.format(INSTANCE)

order_limits = {
    'p' : range(1,1+ 2),
    'd' : range(1,1+ 1),
    'q' : range(1,1+ 2),
    'sp' : range(1,1+ 1),
    'sd' : range(0,1+ 0),
    'sq' : range(1,1+ 1),
    's': [24]
}

auto_sarima(training_data=training_data, order_limits=order_limits, timesteps=timesteps, days=days, ruta=ruta)
    

  0%|          | 0/61 [00:00<?, ?it/s]

 Model: (2, 1, 0), s(0, 0, 0, 0)


  0%|          | 0/61 [00:00<?, ?it/s]

0.056350583045384886
 Model: (2, 1, 0), s(0, 0, 0, 24)


  0%|          | 0/61 [00:00<?, ?it/s]

...

0.03161856552754785
 Model: (2, 1, 2), s(1, 0, 0, 24)


  3%|▎         | 2/61 [00:05<02:32,  2.58s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  5%|▍         | 3/61 [00:07<02:32,  2.63s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  7%|▋         | 4/61 [00:10<02:34,  2.72s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  8%|▊         | 5/61 [00:13<02:35,  2.78s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/ba

0.0314342560535695
 Model: (2, 1, 2), s(1, 0, 1, 24)


  2%|▏         | 1/61 [00:03<03:12,  3.21s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  5%|▍         | 3/61 [00:11<03:50,  3.97s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  7%|▋         | 4/61 [00:15<03:44,  3.94s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  8%|▊         | 5/61 [00:19<03:44,  4.00s/it]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/ba

0.029832881984851397


In [7]:
for i in tqdm(range(len(instances))):

    if i<2:
        continue

    data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[i]))
    data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
    data_series.index = data_series['local_15min']
    data_series.drop(columns=['local_15min'], inplace=True)
    data_demand = data_series[['nonshiftable']]
    data_demand_hourly = data_demand.resample("H").mean()

    start_date = "2019-05-17"
    end_date = "2019-08-01"
    training_data = data_demand_hourly[(data_demand_hourly.index >= start_date) & (data_demand_hourly.index<end_date)]

    timesteps = 24
    days = 15 # 5 dias de observación
    ruta = 'resultados/60min/sarima_15/sarima_i{}.xlsx'.format(i)
    order_limits = {
        'p' : range(1,1+ 2),
        'd' : range(0,1+ 1),
        'q' : range(1,1+ 2),
        'sp' : range(1,1+ 1),
        'sd' : range(0,1+ 0),
        'sq' : range(1,1+ 1),
        's': [24]
    }
   
    auto_sarima(training_data=training_data, order_limits=order_limits, timesteps=timesteps, days=days, ruta=ruta)

  0%|          | 0/14 [00:00<?, ?it/s]/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/juancamilopico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Usi

LinAlgError: LU decomposition error.

In [20]:
# Mejor modelo

# Parámetros
timesteps = 24
days = 31
df = sarima_model(training_data=training_data, order=(1,1,2), seasonal_order=(1,0,1,24), timesteps=timesteps, days=days)
df['mse'] = (df['nonshiftable']-df['forecast'])**2
df['mae'] = abs(df['nonshiftable']-df['forecast'])
df['mape'] = abs(df['nonshiftable']-df['forecast'])/df['nonshiftable']
mse_mean = df['mse'].mean()
mse_sd = df['mse'].var()**(1/2)
mae = df['mae'].mean()
mape = df['mape'].mean()
print('mse:', mse_mean)
print('mse_sd:',mse_sd)
print('mae',np.round(mae,2))
print('mape',np.round(mape,4))



 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -5.57601D-01    |proj g|=  3.65624D-01

At iterate    5    f= -5.61916D-01    |proj g|=  5.27038D-01

At iterate   10    f= -5.80153D-01    |proj g|=  1.86108D-02

At iterate   15    f= -5.82469D-01    |proj g|=  1.33362D-01

At iterate   20    f= -6.04618D-01    |proj g|=  6.00751D-02

At iterate   25    f= -6.22176D-01    |proj g|=  7.17751D-02

At iterate   30    f= -6.31484D-01    |proj g|=  4.64675D-02

At iterate   35    f= -6.32451D-01    |proj g|=  1.31526D-02

At iterate   40    f= -6.32464D-01    |proj g|=  7.42851D-03

At iterate   45    f= -6.32466D-01    |proj g|=  2.54878D-03



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   50    f= -6.32467D-01    |proj g|=  7.61915D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     50     91      2     0     0   7.619D-04  -6.325D-01
  F = -0.63246742112757781     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


 This problem is unconstrained.


...

At iterate    5    f= -6.31033D-01    |proj g|=  3.50618D-01

At iterate   10    f= -6.41678D-01    |proj g|=  8.50260D-01

At iterate   15    f= -6.49196D-01    |proj g|=  3.84510D-02

At iterate   20    f= -6.53936D-01    |proj g|=  7.57618D-02

At iterate   25    f= -6.81398D-01    |proj g|=  2.08436D-02

At iterate   30    f= -6.85396D-01    |proj g|=  7.61361D-03

At iterate   35    f= -6.85840D-01    |proj g|=  6.24605D-02

At iterate   40    f= -6.85884D-01    |proj g|=  7.17979D-04

At iterate   45    f= -6.85999D-01    |proj g|=  6.73533D-03

At iterate   50    f= -6.86060D-01    |proj g|=  2.05990D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf

 This problem is unconstrained.



At iterate   10    f= -6.60344D-01    |proj g|=  2.82131D-01

At iterate   15    f= -6.61256D-01    |proj g|=  6.13979D-02

At iterate   20    f= -6.63731D-01    |proj g|=  3.91615D-01

At iterate   25    f= -6.82329D-01    |proj g|=  1.17791D-01

At iterate   30    f= -6.83705D-01    |proj g|=  5.12350D-02

At iterate   35    f= -6.83765D-01    |proj g|=  2.45833D-03

At iterate   40    f= -6.83793D-01    |proj g|=  3.28467D-03

At iterate   45    f= -6.83991D-01    |proj g|=  7.95192D-02

At iterate   50    f= -6.84055D-01    |proj g|=  3.88010D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     50     59   

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.40910D-01    |proj g|=  3.44957D-01

At iterate    5    f= -6.45664D-01    |proj g|=  4.95053D-01

At iterate   10    f= -6.57733D-01    |proj g|=  4.99372D-01

At iterate   15    f= -6.61895D-01    |proj g|=  7.64490D-02

At iterate   20    f= -6.73748D-01    |proj g|=  6.75239D-01

At iterate   25    f= -6.80479D-01    |proj g|=  2.40509D-01

At iterate   30    f= -6.82878D-01    |proj g|=  5.78626D-03

At iterate   35    f= -6.82932D-01    |proj g|=  1.69443D-03

At iterate   40    f= -6.82961D-01    |proj g|=  2.86746D-02

At iterate   45    f= -6.83883D-01    |proj g|=  2.13173D-01

At iterate   50    f= -6.85029D-01    |proj g|=  5.53622D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

In [21]:
dias = df['Day'].unique()
df['From'] = df['From'].astype(str)
df = df.reset_index(drop=True)
df.head()

,local_15min,nonshiftable,forecast,Upper bound,Lower bound,From,Day,mse,mae,mape
0,2019-06-01 00:00:00,0.055313,0.086360,0.213500,-0.040781,2019-05-31 23:00:00,2019-06-01,0.000964,0.031047,0.561304
1,2019-06-01 01:00:00,0.074063,0.093332,0.236297,-0.049633,2019-05-31 23:00:00,2019-06-01,0.000371,0.019270,0.260183
2,2019-06-01 02:00:00,0.049875,0.110693,0.257478,-0.036092,2019-05-31 23:00:00,2019-06-01,0.003699,0.060818,1.219408
3,2019-06-01 03:00:00,0.112375,0.125111,0.272876,-0.022653,2019-05-31 23:00:00,2019-06-01,0.000162,0.012736,0.113339
4,2019-06-01 04:00:00,0.137750,0.201356,0.349379,0.053333,2019-05-31 23:00:00,2019-06-01,0.004046,0.063606,0.461748


In [22]:
df[df.From=='2019-06-01 00:00:00'].tail()

,local_15min,nonshiftable,forecast,Upper bound,Lower bound,From,Day,mse,mae,mape
42,2019-06-01 19:00:00,0.085687,0.155503,0.303628,0.007378,2019-06-01 00:00:00,2019-06-01,0.004874,0.069815,0.814766
43,2019-06-01 20:00:00,0.126250,0.140803,0.288928,-0.007322,2019-06-01 00:00:00,2019-06-01,0.000212,0.014553,0.115271
44,2019-06-01 21:00:00,0.112313,0.124276,0.272401,-0.023849,2019-06-01 00:00:00,2019-06-01,0.000143,0.011963,0.106519
45,2019-06-01 22:00:00,0.102250,0.105658,0.253783,-0.042467,2019-06-01 00:00:00,2019-06-01,0.000012,0.003408,0.033327
46,2019-06-01 23:00:00,0.094125,0.100508,0.248632,-0.047616,2019-06-01 00:00:00,2019-06-01,0.000041,0.006383,0.067812


In [23]:
df_real= df[['local_15min', 'nonshiftable']].drop_duplicates()

In [39]:
df["From2"] = "Forecast at " + df["From"].str.slice(start=-8, stop=-3)

DIAS = dias[3:4]

colorscale = []
rgb = px.colors.convert_colors_to_same_type(px.colors.sequential.haline)[0]
n_steps = int(np.round(28/(len(rgb)-1)))  # Control the number of colors in the final colorscale
for i in range(len(rgb) - 1):
    for step in np.linspace(0, 1, n_steps):
        colorscale.append(px.colors.find_intermediate_color(rgb[i], rgb[i + 1], step, colortype='rgb'))

for i in range(len(colorscale)):
    colorscale[i] = colorscale[i][:-1]+', 0.5)'
    colorscale[i] = 'rgba'+colorscale[i][3:]

fig = px.line(df[df['Day'].isin(DIAS)], x="local_15min", y="forecast", color='From2',color_discrete_sequence=colorscale, labels={"From2": ""})

labels_to_show_in_legend = ["Forecast at 0{}:00".format(i) for i in range(0, 10, 2)]+["Forecast at {}:00".format(i) for i in range(10, 24, 2)]

for trace in fig['data']: 
    if (not trace['name'] in labels_to_show_in_legend):
        trace['showlegend'] = False

fig.add_trace(go.Scatter(x=df_real[df_real['local_15min'].dt.date.isin(pd.to_datetime(DIAS).date)]["local_15min"], 
                         y=df_real[df_real['local_15min'].dt.date.isin(pd.to_datetime(DIAS).date)]["nonshiftable"], 
                         name="Real energy demand",
                         line=dict(color='black'),))

fig.update_layout(title_text="Rolling forecast of non-shiftable energy demand",
                  yaxis=dict(title="Energy [kWh]"),
                  xaxis=dict(title= "Time"))

fig.write_image("../../Soportes documento/Graficas documento/demand_forecast.png", scale=2, height = 450, width=700)
fig.show()